# GRAD 5100 Final Project Part Three
## The TopChef Database
## Christine Nguyen

### Part Three

**The topChef database**

TopChef is a reality TV show in which contestants, who are professional chefs, compete in a variety of cooking contests over the course of a season. Each episode, one or more chefs are kicked off the show, until, at the end the winner is crowned “Top Chef”.

The topChef database contains information about (most of) the top Chef seasons.

You can access the top chef database using the same credentials we used in class for the sakila database. The only difference is that the database name (dbname) is topChef instead of sakila.

If you would prefer to work in R (and even avoid SQL), you can load the topChef data by install.packages("topChef") and library("topChef").

There are 6 tables in the topChef database but we will focus our attention on two of them: chefdetails and judges. The chefdetails tells us information about the contestants, and the judges table tells us information about the judges.

Of particular interest in the chefdetails table are the fields:

* season which is the name of the season (often a city where the competition took place, but not always)
* seasonNumber which is the number of the season
* name which is the chef’s name
* placement which tells where the chef finished in the competition
* gender which tells the chef’s gender.

In the judges table we are interested in:

* season and seasonNumber which are the same as in the chefdetails table.
* episode which gives the episode where the judge appeared; the show uses many guest judges varying episode by episode.
* challengeType describes the type of challenge where the judge played a role.
* guestJudge is the judge’s name
* competedOnTC is ‘Yes’ if the judge was a previous top chef contestant.

Top Chef problems

1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series. The database contains information on both. We are not interested in the ‘Masters’. There is also a season called “Canada 6” which is part of the Masters series. Create versions of the chefdetails and judges tables that exclude any records coming from seasons that include the word Masters or Canada.

2. Further trim your chefdetails and judges tables by including only the columns of interest from the table descriptions above.

3. Your tables should have information from 20 remaining different seasons, numbered from 1 to 20. Make a table that has two columns, one called season and one called seasonNumber showing the number associated with each season. (For example, San Francisco is season 1.)

4. Answer the following questions using your tables so far.
    1. Among all chef contestants, how many are male and how many are female?
    2. Among all winners (placement==1) how many are male and how many are female?
    3. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female?
    4. Compare the number of times a female was placed in the top 3 with the number of times a female won. Does this number seem unusual? That is, are women who place in the top 3 less likely than expected to ultimately win the competition? Why or why not?
5. Make a table with one row for each first place winner containing the winner’s name, the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they served as a guest judge or not.
6. Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.

In [1]:
import sqlalchemy as sqla
import pymysql
import pandas as pd
import os

username = "user"
password = "grad5100user"
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
port = 3306
dbname = "topChef"

dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

In [2]:
def run_sql(sql, engine):
    "send sql to database given by engine and return the result"
    with engine.connect() as conn:
        result = conn.execute(sqla.text(sql))
    return result.all()

def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

#### Task 1 and Task 2.

In [3]:
chefdetails_query = "SELECT * FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season NOT LIKE 'Canada%' AND season != 'Canada 6'"
judges_query = "SELECT * FROM judges WHERE season NOT LIKE '%Masters%' AND season NOT LIKE 'Canada%' AND season != 'Canada 6'"

chefdetails_query = "SELECT season, seasonNumber, name, placement, gender FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6'"
judges_query = "SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC FROM judges WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6'"

chefdetails_df = df_select(chefdetails_query, engine)
print("chefdetails_df:")
print(chefdetails_df.head())

judges_df = df_select(judges_query, engine)
print("judges_df:")
print(judges_df.head())

Retrieved 322 records
chefdetails_df:
                season  seasonNumber            name  placement  gender
0  All Stars: New York             8   Richard Blais          1    Male
1  All Stars: New York             8   Mike Isabella          2    Male
2  All Stars: New York             8  Antonia Lofaso          3  Female
3  All Stars: New York             8   Tiffany Derry          4  Female
4  All Stars: New York             8      Carla Hall          5  Female
Retrieved 633 records
judges_df:
                season  seasonNumber  episode challengeType        guestJudge  \
0  All Stars: New York             8        1     Quickfire     Tom Colicchio   
1  All Stars: New York             8        1   Elimination  Anthony Bourdain   
2  All Stars: New York             8        2     Quickfire         Joe Jonas   
3  All Stars: New York             8        2   Elimination         Katie Lee   
4  All Stars: New York             8        3     Quickfire       David Chang   

  competed

#### Task 3.

In [4]:
seasons_query = "SELECT DISTINCT season, seasonNumber FROM chefdetails WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6'"

seasons_df = df_select(seasons_query, engine)
print("seasons_df:")
print(seasons_df)

Retrieved 20 records
seasons_df:
                 season  seasonNumber
0   All Stars: New York             8
1        All-Stars L.A.            17
2                Boston            12
3            California            13
4            Charleston            14
5               Chicago             4
6                  D.C.             7
7              Colorado            15
8               Houston            19
9              Kentucky            16
10            Las Vegas             6
11          Los Angeles             2
12                Miami             3
13          New Orleans            11
14             New York             5
15             Portland            18
16        San Francisco             1
17              Seattle            10
18                Texas             9
19      World All Stars            20


#### Task 4a.

In [5]:
gender_count_query = """
SELECT gender, COUNT(*) as count
FROM chefdetails
WHERE season NOT LIKE '%Masters%' AND season != 'Canada 6'
GROUP BY gender
"""

gender_count_df = df_select(gender_count_query, engine)
print("gender_count_df:")
print(gender_count_df)

Retrieved 2 records
gender_count_df:
   gender  count
0    Male    170
1  Female    152


#### Task 4b.

In [6]:
winners_gender_count_query = """
SELECT gender, COUNT(*) as count
FROM chefdetails
WHERE placement = 1 AND season NOT LIKE '%Masters%' AND season != 'Canada 6'
GROUP BY gender
"""

winners_gender_count_df = df_select(winners_gender_count_query, engine)
print("winners_gender_count_df:")
print(winners_gender_count_df)

Retrieved 2 records
winners_gender_count_df:
   gender  count
0    Male     14
1  Female      6


#### Task 4c.

In [7]:
top3_gender_count_query = """
SELECT gender, COUNT(*) as count
FROM chefdetails
WHERE placement IN (1, 2, 3) AND season NOT LIKE '%Masters%' AND season != 'Canada 6'
GROUP BY gender
"""

top3_gender_count_df = df_select(top3_gender_count_query, engine)
print("top3_gender_count_df:")
print(top3_gender_count_df)

Retrieved 2 records
top3_gender_count_df:
   gender  count
0    Male     36
1  Female     25


#### Task 4d.

**Answer**: The total number of times a female chef reached the top 3 is 25, and the total number of times a female won is 6. This situation seems unusual, as the ratio of females placed in the top 3 to those who won is 25:6, or approximately 4.17:1. For every roughly 4 times a female chef reaches the top 3, 1 of them wins. The result suggests that women who place in the top 3 are less likely than expected to ultimately win the competition. This is based on the ratio of 4.17:1, indicating that reaching the top 3 does not guarantee a victory for female contestants.

In [8]:
compare_female_query = """
SELECT
  (SELECT COUNT(*) FROM chefdetails WHERE placement IN (1, 2, 3) AND gender = 'Female' AND season NOT LIKE '%Masters%' AND season != 'Canada 6') as top3_count,
  (SELECT COUNT(*) FROM chefdetails WHERE placement = 1 AND gender = 'Female' AND season NOT LIKE '%Masters%' AND season != 'Canada 6') as winner_count
"""

compare_female_df = df_select(compare_female_query, engine)
print("compare_female_df:")
print(compare_female_df)

Retrieved 1 records
compare_female_df:
   top3_count  winner_count
0          25             6


#### Task 5.

In [9]:
first_place_guest_judges_query = """
SELECT
  winner_name,
  winning_season,
  CASE WHEN EXISTS (
    SELECT 1 FROM judges
    WHERE guestJudge = first_place_winners.winner_name OR competedOnTC = first_place_winners.winner_name
  ) THEN 'Yes' ELSE 'No' END as served_as_guest_judge
FROM
  (
    SELECT
      cd.name as winner_name,
      cd.season as winning_season
    FROM
      chefdetails cd
    WHERE
      cd.placement = 1
      AND cd.season NOT LIKE '%Masters%' AND cd.season != 'Canada 6'
  ) as first_place_winners;
""" 

first_place_guest_judges_query_df = df_select(first_place_guest_judges_query, engine)

print("first_place_guest_judges_query_df:")
print(first_place_guest_judges_query_df)

Retrieved 20 records
first_place_guest_judges_query_df:
             winner_name       winning_season served_as_guest_judge
0          Richard Blais  All Stars: New York                   Yes
1           Melissa King       All-Stars L.A.                   Yes
2                Mei Lin               Boston                    No
3            Jeremy Ford           California                    No
4      Brooke Williamson           Charleston                   Yes
5        Stephanie Izard              Chicago                   Yes
6           Kevin Sbraga                 D.C.                    No
7           Joseph Flamm             Colorado                    No
8             Kah-wai Lo              Houston                    No
9   Kelsey Barnard Clark             Kentucky                   Yes
10     Michael Voltaggio            Las Vegas                   Yes
11             Ilan Hall          Los Angeles                   Yes
12            Hung Huynh                Miami               

#### Task 6.

In [13]:
non_winners_guest_judges_query = """
SELECT DISTINCT
  non_winner_name,
  'Yes' as served_as_guest_judge
FROM
  (
    SELECT
      cd.name as non_winner_name
    FROM
      chefdetails cd
    WHERE
      cd.placement != 1
      AND cd.season NOT LIKE '%Masters%' AND cd.season != 'Canada 6'
  ) as non_winner_guests
WHERE
  EXISTS (
    SELECT 1 FROM judges
    WHERE guestJudge = non_winner_guests.non_winner_name OR competedOnTC = non_winner_guests.non_winner_name
  );
"""

non_winners_guest_judges_df = df_select(non_winners_guest_judges_query, engine)


print("non_winners_guest_judges_df:")
print(non_winners_guest_judges_df)

Retrieved 32 records
non_winners_guest_judges_df:
       non_winner_name served_as_guest_judge
0        Mike Isabella                   Yes
1       Antonia Lofaso                   Yes
2        Tiffany Derry                   Yes
3           Dale Talde                   Yes
4     Spike Mendelsohn                   Yes
5     Stephen Asprinio                   Yes
6      Bryan Voltaggio                   Yes
7      Gregory Gourdet                   Yes
8      Karen Akunowicz                   Yes
9        Lee Anne Wong                   Yes
10         Nini Nguyen                   Yes
11        Melissa King                   Yes
12        Amar Santana                   Yes
13      Kwame Onwuachi                   Yes
14      Sheldon Simeon                   Yes
15          Sam Talbot                   Yes
16       Richard Blais                   Yes
17   Adrienne Cheatham                   Yes
18        Carrie Baird                   Yes
19     Michael Midgley                   Yes
20   